# Z3 Solver Project Demo
Presentation purposes

*Svolto per la presentazione del progetto all'orale*

In [1]:
import pandas as pd 
import numpy as np 
import itertools 
from model import TimetableScheduler

## Scenario 1: Toy Model
Simple model containing just a few sessions and courses, not intended to be realistic in any way

*Modello semplice che contiene qualche sessione e corsi, inteso solo per motivi di testing*

In [2]:
toy_schedule = TimetableScheduler('sample_1', 8)

In [3]:
toy_schedule.start()

In [ ]:
toy_schedule.add_constraints()

In [ ]:
toy_schedule.solve()

TIME TABLE SUCCESSFULLY CREATED


In [65]:
timetable = []

In [66]:
names = toy_schedule.database.get_names()

In [67]:
# stampa l'orario per AIDA

for t in toy_schedule.T:
    t_slots = []

    for (S, A) in itertools.product(toy_schedule.indexes['Sessions'], toy_schedule.indexes['Rooms']):
        S_c = toy_schedule.database.get_course(S)
        if toy_schedule.model.evaluate(toy_schedule.X[S,t,A], model_completion=True):
            t_slots.append(f"{names['Courses'][toy_schedule.database.get_course(S)]} [{names['Rooms'][A]}]")
                
    if t_slots == []:
        t_slots = ["NESSUNA LEZIONE"]
    timetable.append("\n".join(t_slots))

In [68]:
rows = ["08:00-09:00","09:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00"]
columns = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]

In [69]:
np_timetable = np.array(timetable, dtype=object)
np_timetable = np_timetable.reshape(6,8)

In [70]:
df = pd.DataFrame(np_timetable, columns=rows, index=columns)

In [72]:
from IPython.display import display, HTML # per fare una presentazione grafica decente
display(HTML(df.T.to_html().replace("\\n", "<br>")))

,Lunedì,Martedì,Mercoledì,Giovedì,Venerdì,Sabato
08:00-09:00,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],ANALISI 2 [Aula Morin 3],ANALISI 2 [Aula Magna H3],ANALISI 1 [Aula Ciamician],ANALISI 1 [Aula Ciamician]
09:00-10:00,SISTEMI OPERATIVI [Aula Ciamician]GEOMETRIA [Aula Morin],PROGRAMMAZIONE [Aula Ciamician],PROGRAMMAZIONE [Aula Ciamician]ANALISI 2 [Aula Morin 3],ANALISI 2 [Aula Magna H3],ANALISI 1 [Aula Ciamician],ANALISI 1 [Aula Ciamician]
10:00-11:00,SISTEMI OPERATIVI [Aula Ciamician]GEOMETRIA [Aula Morin],PROGRAMMAZIONE [Aula Ciamician],PROGRAMMAZIONE [Aula Ciamician]ANALISI 2 [Aula Morin 3],ANALISI 2 [Aula Magna H3],NESSUNA LEZIONE,ANALISI 1 [Aula Ciamician]
11:00-12:00,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],PROGRAMMAZIONE [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin 5]ANALISI 2 [Aula Magna H3]
12:00-13:00,NESSUNA LEZIONE,GEOMETRIA [Aula Morin 6],ANALISI 1 [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin 5]ANALISI 2 [Aula Magna H3]
13:00-14:00,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin 5]GEOMETRIA [Aula Morin 6],ANALISI 1 [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin]ANALISI 2 [Aula Magna H3]
14:00-15:00,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin 5]GEOMETRIA [Aula Morin 6],ANALISI 1 [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,SISTEMI OPERATIVI [Aula Morin]
15:00-16:00,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE


## Scenario 2: Next Semester's Courses for AIDA
Simplified model containing (estimated) sessions and courses for the next semester in AIDA.

*Modello semplificato che contiene i corsi del prossimo semestre del CdS triennale in IADA (2025-2026, Primo Semestre)*

Vincolo aggiuntivo: Non ci sono lezioni il sabato
$$
\forall S, T, A, \text{ s.t. }T\text{ rappresenta un'ora in sabato }(\lfloor T/6 \rfloor = 6), X_{S,T,A} \implies \text{False}
$$

In [73]:
orario_aida = TimetableScheduler('aida', 9)

In [74]:
orario_aida.start()

In [75]:
from z3 import Not

In [76]:
# add custom constraint
for (S,T,A) in itertools.product(orario_aida.indexes['Sessions'],
                                 orario_aida.T,
                                 orario_aida.indexes['Rooms']):
    if (T // 9) == (6-1):
        orario_aida.solver.add(Not(orario_aida.X[S,T,A]))


In [77]:
from z3 import Then, sat

Then('simplify', 'propagate-values', 'solve-eqs', 'smt')
sat.local_search = True

In [78]:
orario_aida.add_constraints()
orario_aida.solve()

TIME TABLE SUCCESSFULLY CREATED


In [133]:
# stampa l'orario per AIDA
names = orario_aida.database.get_names()
aida_timeslots = []

for t in orario_aida.T:
    t_slots = []

    for (S, A) in itertools.product(orario_aida.indexes['Sessions'], orario_aida.indexes['Rooms']):
        S_c = orario_aida.database.get_course(S)
        if orario_aida.model.evaluate(orario_aida.X[S,t,A], model_completion=True):
            t_slots.append(f"{names['Courses'][orario_aida.database.get_course(S)]} [{names['Rooms'][A]}]")
                
    if t_slots == []:
        t_slots = ['NESSUNA LEZIONE']
    aida_timeslots.append(" > " + "\n > ".join(t_slots))

In [134]:
rows = ["09:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00","16:00-17:00","17:00-18:00"]
columns = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]

np_aida_timetable = np.array(aida_timeslots, dtype=object)
np_aida_timetable = np_aida_timetable.reshape(6,9)
df = pd.DataFrame(np_aida_timetable, columns=rows, index=columns)

In [135]:
display(HTML(df.T.to_html().replace("\\n", "<br>")))

,Lunedì,Martedì,Mercoledì,Giovedì,Venerdì,Sabato
09:00-10:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H2 Bis, 3B] > SISTEMI COMPLESSI [H2 Bis, 2A MORIN]","> ANALISI 1 [H2 Bis, 2A MORIN] > PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B]","> ANALISI 1 [H3, 1B] > INFERENZA STATISTICA [H2 Bis, 3B]",> NESSUNA LEZIONE
10:00-11:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H2 Bis, 3B] > SISTEMI COMPLESSI [H2 Bis, 2A MORIN]","> ANALISI 1 [H2 Bis, 2A MORIN] > PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B]","> ANALISI 1 [H3, 1B] > INFERENZA STATISTICA [H2 Bis, 3B]",> NESSUNA LEZIONE
11:00-12:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H3, 2A] > SISTEMI COMPLESSI [H2 Bis, 2A MORIN]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > INFERENZA STATISTICA [H3, 1C] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 3B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A]","> ARCHITETTURE DEGLI ELABORATORI [H3, 2B]",> NESSUNA LEZIONE,> NESSUNA LEZIONE
12:00-13:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H3, 2A]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > INFERENZA STATISTICA [H3, 1C] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 3B]","> INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H3, 2A]","> ARCHITETTURE DEGLI ELABORATORI [H3, 2B] > INFERENZA STATISTICA [H3, 2C] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 2A MORIN]","> COMPUTABILITà, COMPLESSITà E LOGICA [H2 Bis, 3B]",> NESSUNA LEZIONE
13:00-14:00,"> PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 4C]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > ALGORITMI E STRUTTURE DATI [H3, 2B] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H3, 2A]","> ARCHITETTURE DEGLI ELABORATORI [H3, 2B] > INFERENZA STATISTICA [H3, 2C] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 2A MORIN]","> COMPUTABILITà, COMPLESSITà E LOGICA [H2 Bis, 3B]",> NESSUNA LEZIONE
14:00-15:00,"> PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 4C]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > ALGORITMI E STRUTTURE DATI [H3, 2B] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H3, 2A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H2 Bis, 2A MORIN] > INTRODUZIONE AL MACHINE LEARNING [H3, 1A]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 2A]",> NESSUNA LEZIONE
15:00-16:00,"> ANALISI 1 [H2 Bis, 2A MORIN] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5C]","> COMPUTABILITà, COMPLESSITà E LOGICA [H3, 1C] > SISTEMI COMPLESSI [H3, 1A]",> NESSUNA LEZIONE,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H2 Bis, 2A MORIN] > INTRODUZIONE AL MACHINE LEARNING [H3, 1A]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 2A] > PROGRAMMAZIONE AVANZATA E PARALLELA [H2 Bis, 2A MORIN]",> NESSUNA LEZIONE
16:00-17:00,"> ANALISI 1 [H2 Bis, 2A MORIN] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5C]","> COMPUTABILITà, COMPLESSITà E LOGICA [H3, 1C] > SISTEMI COMPLESSI [H3, 1A]",> NESSUNA LEZIONE,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B]","> PROGRAMMAZIONE AVANZATA E PARALLELA [H2 Bis, 2A MORIN]",> NESSUNA LEZIONE
17:00-18:00,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE


In [136]:
df.to_excel("./timetables/aida.xlsx")

## Scenario 3: High School Timetable
Model creating a time table for the lessons of a class in high school, the last 3 years

*Modello che crea un orario settimanale per le lezioni del trienno di un liceo linguistico, solo per una classe (per semplicità assumiamo che tutti i professori siano assegnati ad una e sola classe)*